In [1]:
#!pip3 install dwave-system

In [2]:
import pandas as pd 
import numpy as np
from scipy.special import comb
import math
from neal import SimulatedAnnealingSampler
import itertools
import random
import matplotlib.pyplot as plt
from dwave.system.samplers import DWaveSampler
from dwave.system.composites import EmbeddingComposite
from dwave.embedding.chain_strength import uniform_torque_compensation
import dimod
import timeit

In [3]:
def calc_marginals(df):                   
    return np.array([                      
        sum(df['Y']),                     
        np.dot(df['Y'], df['LI']),      
        np.dot(df['Y'], df['SEX']),      
        np.dot(df['Y'], df['AOP']),      
    ])

In [4]:
def make_Hamiltonian(df, t1):
    t_list = calc_marginals(df)
    N=len(df)
    dup_list = [(i, i) for i in range(N)]
    comb_list = [(i, j) for i in range(N) for j in range(i+1, N)]
    
    lin_Y = [1-2*t_list[0] for (i, _) in dup_list] #同じy同士
    quad_Y = [2 for (i, j) in comb_list] #異なるy同士
    num_Y = t_list[0]**2 #数字の二乗
    
    LI = df['LI'].iloc
    lin_LI = [(LI[i] - 2 * t1) * LI[i] for (i, _) in dup_list]
    quad_LI = [2*LI[i] * LI[j] for (i, j) in comb_list]
    num_LI = t1**2
    
    SEX = df['SEX'].iloc
    lin_SEX  = [(SEX[i] - 2 * t_list[2]) * SEX[i] for (i, _) in dup_list]
    quad_SEX  = [2*SEX[i] * SEX[j] for (i, j) in comb_list]
    num_SEX  = t_list[2]**2
    
    AOP = df['AOP'].iloc
    lin_AOP = [(AOP[i] - 2 * t_list[3]) * AOP[i] for (i, _) in dup_list]
    quad_AOP = [2*AOP[i] * AOP[j] for (i, j) in comb_list]
    num_AOP = t_list[3]**2
    
    #lin
    lin_list = [sum(lin) for lin in zip(lin_Y, lin_LI, lin_SEX, lin_AOP)]
    lin = {i: lin_list[i] for (i, _) in dup_list}
    
    #quad
    quad_values = [sum(quad) for quad in zip(quad_Y, quad_LI, quad_SEX, quad_AOP)]
    quad = {ij: quad_values[n] for (n, ij) in enumerate(comb_list)}
    
    #num
    num = num_Y + num_LI + num_SEX + num_AOP
    
    #print('lin:', lin)
    #print('quad:', quad)
    #print('num:', num)
    return dimod.BinaryQuadraticModel(lin, quad, num, dimod.Vartype.BINARY)#dic, dic, num

In [25]:
def find_valid_y_1(df, num_reads):                                                        
    
    # dimod sampler
    dw_sampler = DWaveSampler(
        endpoint="https://cloud.dwavesys.com/sapi",
        solver = 'DW_2000Q_6',
        token = "TOKY-1319d5c52b9aa35f34b40feba0cea58a4f5d3c09"
    )
    
    # インスタンス作成
    qa_sampler = EmbeddingComposite(dw_sampler)
    
    t_list = calc_marginals(df)
    valid_y_list = {}                                                                   
    valid_y_num = {}
    
    for t1 in range(0, sum(df['LI'])+1):                                                                                                                                                                                                                                               
        bqm = make_Hamiltonian(df, t1)
        
        res = qa_sampler.sample(bqm, chain_break_fraction=True, num_reads=num_reads)
        
        valid_y_list[t1] = []                                                           
        valid_y_num[t1] = 0                                                             
        for y_info in list(res.record):
            #print(y_info)
            if y_info[1] == 0.0:
                valid_y_num[t1] += 1                                        
                valid_y_list[t1].append(list(y_info[0]))                    
                #print('energy0')
                                                      
    return valid_y_list, valid_y_num       

In [48]:
def find_valid_y_10(df, num_reads):                                                        
    
    # dimod sampler
    dw_sampler = DWaveSampler(
        endpoint="https://cloud.dwavesys.com/sapi",
        solver = 'DW_2000Q_6',
        token = "TOKY-1319d5c52b9aa35f34b40feba0cea58a4f5d3c09"
    )
    
    # インスタンス作成
    qa_sampler = EmbeddingComposite(dw_sampler)
    
    t_list = calc_marginals(df)
    valid_y_list = {}                                                                   
    valid_y_num = {}
    
    for t1 in range(0, sum(df['LI'])+1):                                                                                                                                                                                                                                               
        Q = make_Hamiltonian(df, t1)
        
        res = qa_sampler.sample(Q, chain_break_fraction=True, num_reads=num_reads)
        
        valid_y_list[t1] = []                                                           
        valid_y_num[t1] = 0                                                                     
        for (sample, energy, num, chain_strength) in res.data():
            sample_list = list(sample.values())
            print("Sample: ", sample_list, "Energy: ", energy, "Occurrences: ", num)
            if energy == 0.0:
                print('ok!')
                valid_y_num[t1] += 1                                        
                valid_y_list[t1].append(sample_list)         
                                                      
    return 0   

In [23]:
def find_valid_y_1_check(df, num_reads):                                                        
    
    # dimod sampler
    dw_sampler = DWaveSampler(
        endpoint="https://cloud.dwavesys.com/sapi",
        solver = 'DW_2000Q_6',
        token = "TOKY-1319d5c52b9aa35f34b40feba0cea58a4f5d3c09"
    )
    
    # インスタンス作成
    qa_sampler = EmbeddingComposite(dw_sampler)
    
    t_list = calc_marginals(df)
    
    for t1 in range(0, sum(df['LI'])+1):                                                                                                                                                                                                                                               
        Q = make_Hamiltonian(df, t1)
        
        res = qa_sampler.sample(Q, chain_break_fraction=True, num_reads=num_reads)
        
        for (sample, energy, num, chain) in res.data():
            sample_list = list(sample.values())
            print("Sample: ", sample_list, "Energy: ", energy, "Occurrences: ", num)
                                                      
    return 0       

In [29]:
def find_valid_y_2(df, num_reads):                                                        
    
    # dimod sampler
    dw_sampler = DWaveSampler(
        endpoint="https://cloud.dwavesys.com/sapi",
        solver = 'DW_2000Q_6',
        token = "TOKY-1319d5c52b9aa35f34b40feba0cea58a4f5d3c09"
    )
    
    # インスタンス作成
    qa_sampler = EmbeddingComposite(dw_sampler)
    
    t_list = calc_marginals(df)
    valid_y_list = {}                                                                   
    valid_y_num = {}
    
    for t1 in range(0, sum(df['LI'])+1):                                                                                                                                                                                                                                               
        bqm = make_Hamiltonian(df, t1)
        
        res = qa_sampler.sample(bqm, chain_break_fraction=False, num_reads=num_reads)
        
        valid_y_list[t1] = []                                                           
        valid_y_num[t1] = 0                                                             
        for y_info in list(res.record):
            print(y_info)
            if y_info[1] == 0.:
                valid_y_num[t1] += 1                                        
                valid_y_list[t1].append(list(y_info[0]))                    
                #print('energy0')
                                                      
    return valid_y_list, valid_y_num       

In [42]:
def find_valid_y_20(df, num_reads):                                                        
    
    # dimod sampler
    dw_sampler = DWaveSampler(
        endpoint="https://cloud.dwavesys.com/sapi",
        solver = 'DW_2000Q_6',
        token = "TOKY-1319d5c52b9aa35f34b40feba0cea58a4f5d3c09"
    )
    
    # インスタンス作成
    qa_sampler = EmbeddingComposite(dw_sampler)
    
    t_list = calc_marginals(df)
    valid_y_list = {}                                                                   
    valid_y_num = {}
    
    for t1 in range(0, sum(df['LI'])+1):                                                                                                                                                                                                                                               
        Q = make_Hamiltonian(df, t1)
        
        res = qa_sampler.sample(Q, chain_break_fraction=False, num_reads=num_reads)
        
        valid_y_list[t1] = []                                                           
        valid_y_num[t1] = 0                                                                     
        for (sample, energy, num) in res.data():
            sample_list = list(sample.values())
            print("Sample: ", sample_list, "Energy: ", energy, "Occurrences: ", num)
            if energy == 0.0:
                print('ok!')
                valid_y_num[t1] += 1                                        
                valid_y_list[t1].append(sample_list)         
                                                      
    return 0       

In [7]:
def find_valid_y_3(df, num_reads):                                                        
    # dimod sampler
    dw_sampler = DWaveSampler(
        endpoint="https://cloud.dwavesys.com/sapi",
        solver = 'DW_2000Q_6',
        token = "TOKY-1319d5c52b9aa35f34b40feba0cea58a4f5d3c09"
    )
    
    # インスタンス作成
    qa_sampler = EmbeddingComposite(dw_sampler)
    
    t_list = calc_marginals(df)
    valid_y_list = {}                                                                   
    valid_y_num = {}
    
    t1 = 6
    bqm = make_Hamiltonian(df, t1)

    for chain_strength in range(0, 100):
        res = qa_sampler.sample(bqm, chain_strength = chain_strength, chain_break_fraction=True, num_reads=num_reads)

        valid_y_list[chain_strength] = []                                                           
        valid_y_num[chain_strength] = 0                                                             
        for y_info in list(res.record):
            if y_info[1] == 0.:
                valid_y_num[chain_strength] += 1                                        
                valid_y_list[chain_strength].append(list(y_info[0]))
                                                      
    return valid_y_list, valid_y_num

In [50]:
def find_valid_y_3_check(df, num_reads, chain_strength_max):                                                        
    
    # dimod sampler
    dw_sampler = DWaveSampler(
        endpoint="https://cloud.dwavesys.com/sapi",
        solver = 'DW_2000Q_6',
        token = "TOKY-1319d5c52b9aa35f34b40feba0cea58a4f5d3c09"
    )
    
    # インスタンス作成
    qa_sampler = EmbeddingComposite(dw_sampler)
    
    t_list = calc_marginals(df)
    
    for t1 in range(0, sum(df['LI'])+1):                                                                                                                                                                                                                                               
        Q = make_Hamiltonian(df, t1)
        
        for chain_strength in range(0, chain_strength_max):
            res = qa_sampler.sample(Q, chain_strength = chain_strength, chain_break_fraction=True, num_reads=num_reads)
        
            for (sample, energy, num, chain) in res.data():
                sample_list = list(sample.values())
                print("Sample: ", sample_list, "Energy: ", energy, "Occurrences: ", num)
                                                      
    return 0       

In [17]:
# dimod sampler
dw_sampler = DWaveSampler(
    endpoint="https://cloud.dwavesys.com/sapi",
    solver = 'DW_2000Q_6',
    token = "TOKY-1319d5c52b9aa35f34b40feba0cea58a4f5d3c09"
)
    
# インスタンス作成
qa_sampler = EmbeddingComposite(dw_sampler)

df = pd.read_csv('../../input/ost16.csv', sep=',',index_col=0)
Q = make_Hamiltonian(df, 4)

response = qa_sampler.sample(Q, num_reads=10)
for (sample, energy, num, chain) in response.data():
    sample_list = list(sample.values())
    print("Sample: ", sample_list, "Energy: ", energy, "Occurrences: ", num)

Sample:  [0, 1, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 1, 1, 0] Energy:  3.0 Occurrences:  2
Sample:  [0, 1, 0, 0, 0, 0, 1, 1, 1, 0, 1, 0, 0, 1, 1, 0] Energy:  3.0 Occurrences:  1
Sample:  [0, 1, 0, 1, 0, 0, 1, 1, 1, 0, 1, 0, 0, 1, 0, 0] Energy:  4.0 Occurrences:  1
Sample:  [0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 1, 1, 0] Energy:  6.0 Occurrences:  1
Sample:  [1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0] Energy:  6.0 Occurrences:  1
Sample:  [0, 1, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0] Energy:  8.0 Occurrences:  1
Sample:  [0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 1, 1, 0] Energy:  9.0 Occurrences:  1
Sample:  [1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1] Energy:  15.0 Occurrences:  1
Sample:  [0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1] Energy:  18.0 Occurrences:  1


## 16bit

In [26]:
df = pd.read_csv('../../input/ost16.csv', sep=',',index_col=0)

In [27]:
calc_marginals(df)

array([8, 6, 4, 4])

In [30]:
num_reads = 100
find_valid_y_1(df, num_reads)#embedding >> Autoembedding, t1=7で多い

({0: [],
  1: [],
  2: [],
  3: [],
  4: [],
  5: [],
  6: [],
  7: [],
  8: [],
  9: [],
  10: [],
  11: [],
  12: [],
  13: [],
  14: []},
 {0: 0,
  1: 0,
  2: 0,
  3: 0,
  4: 0,
  5: 0,
  6: 0,
  7: 0,
  8: 0,
  9: 0,
  10: 0,
  11: 0,
  12: 0,
  13: 0,
  14: 0})

In [49]:
num_reads = 100
find_valid_y_10(df, num_reads)#chain_break_fractionはtrue

Sample:  [1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0] Energy:  20.0 Occurrences:  1
Sample:  [1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0] Energy:  20.0 Occurrences:  1
Sample:  [0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0] Energy:  20.0 Occurrences:  3
Sample:  [1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0] Energy:  20.0 Occurrences:  1
Sample:  [1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0] Energy:  20.0 Occurrences:  1
Sample:  [1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0] Energy:  20.0 Occurrences:  2
Sample:  [1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0] Energy:  21.0 Occurrences:  1
Sample:  [1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0] Energy:  21.0 Occurrences:  1
Sample:  [1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0] Energy:  21.0 Occurrences:  1
Sample:  [1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0] Energy:  21.0 Occurrences:  1
Sample:  [0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 1, 0, 0, 1, 0, 0] Energy:  21.0 Occurrences:  1
Sample:  [1, 0, 1, 0, 1, 0, 0, 0

Sample:  [1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0] Energy:  10.0 Occurrences:  1
Sample:  [1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0] Energy:  10.0 Occurrences:  1
Sample:  [1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0] Energy:  10.0 Occurrences:  1
Sample:  [1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 1, 1, 0] Energy:  10.0 Occurrences:  1
Sample:  [1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 1, 1, 0] Energy:  10.0 Occurrences:  1
Sample:  [1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0] Energy:  10.0 Occurrences:  1
Sample:  [1, 1, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0] Energy:  11.0 Occurrences:  1
Sample:  [1, 1, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0] Energy:  11.0 Occurrences:  1
Sample:  [1, 1, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0] Energy:  11.0 Occurrences:  1
Sample:  [1, 1, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0] Energy:  11.0 Occurrences:  1
Sample:  [1, 1, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0] Energy:  11.0 Occurrences:  16
Sample:  [1, 1, 0, 0, 0, 1, 0, 

Sample:  [0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 0] Energy:  2.0 Occurrences:  2
Sample:  [1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0] Energy:  2.0 Occurrences:  1
Sample:  [1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 1, 1, 0] Energy:  3.0 Occurrences:  1
Sample:  [0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0] Energy:  3.0 Occurrences:  1
Sample:  [0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 1, 1, 0] Energy:  4.0 Occurrences:  1
Sample:  [1, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 1, 1, 0] Energy:  5.0 Occurrences:  1
Sample:  [1, 0, 0, 0, 0, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0] Energy:  6.0 Occurrences:  1
Sample:  [0, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 1, 1, 0] Energy:  6.0 Occurrences:  1
Sample:  [1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0] Energy:  6.0 Occurrences:  1
Sample:  [1, 0, 0, 0, 1, 0, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0] Energy:  6.0 Occurrences:  1
Sample:  [1, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 1, 1, 0] Energy:  6.0 Occurrences:  1
Sample:  [0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 

Sample:  [0, 1, 1, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 1, 0, 1] Energy:  2.0 Occurrences:  5
Sample:  [0, 1, 1, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 1, 0, 1] Energy:  2.0 Occurrences:  1
Sample:  [0, 1, 1, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 1, 0, 1] Energy:  2.0 Occurrences:  1
Sample:  [0, 1, 1, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 1, 0, 1] Energy:  2.0 Occurrences:  1
Sample:  [0, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 1, 0, 1] Energy:  2.0 Occurrences:  1
Sample:  [1, 0, 0, 1, 1, 0, 0, 1, 0, 1, 1, 0, 0, 0, 1, 0] Energy:  2.0 Occurrences:  2
Sample:  [0, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 1, 0, 1] Energy:  2.0 Occurrences:  1
Sample:  [0, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 1, 0, 1] Energy:  2.0 Occurrences:  1
Sample:  [0, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 1, 0, 1] Energy:  2.0 Occurrences:  1
Sample:  [0, 1, 1, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 1, 0, 1] Energy:  2.0 Occurrences:  1
Sample:  [0, 1, 1, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 1, 0, 1] Energy:  2.0 Occurrences:  1
Sample:  [0, 1, 1, 0, 0, 1, 1, 1, 0, 0, 0, 

Sample:  [0, 1, 0, 0, 1, 0, 1, 0, 1, 1, 0, 1, 0, 1, 0, 1] Energy:  2.0 Occurrences:  1
Sample:  [0, 0, 0, 0, 1, 0, 1, 1, 1, 1, 0, 1, 0, 0, 0, 1] Energy:  3.0 Occurrences:  1
Sample:  [0, 0, 0, 0, 1, 0, 1, 1, 1, 1, 0, 1, 0, 0, 0, 1] Energy:  3.0 Occurrences:  1
Sample:  [0, 1, 0, 0, 1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 1] Energy:  3.0 Occurrences:  1
Sample:  [0, 1, 0, 0, 1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 1] Energy:  3.0 Occurrences:  1
Sample:  [0, 0, 0, 0, 1, 0, 1, 1, 1, 1, 0, 1, 0, 0, 0, 1] Energy:  3.0 Occurrences:  2
Sample:  [0, 1, 0, 0, 1, 0, 1, 1, 1, 1, 0, 1, 0, 0, 0, 1] Energy:  4.0 Occurrences:  2
Sample:  [0, 1, 0, 0, 1, 0, 1, 1, 1, 1, 0, 1, 0, 0, 0, 1] Energy:  4.0 Occurrences:  1
Sample:  [0, 1, 0, 0, 1, 0, 1, 1, 0, 1, 0, 1, 0, 0, 0, 1] Energy:  4.0 Occurrences:  1
Sample:  [0, 1, 0, 0, 1, 0, 1, 1, 1, 1, 0, 1, 0, 0, 0, 1] Energy:  4.0 Occurrences:  2
Sample:  [0, 1, 0, 0, 1, 0, 1, 1, 1, 1, 0, 1, 0, 0, 0, 1] Energy:  4.0 Occurrences:  1
Sample:  [0, 1, 0, 0, 1, 0, 1, 1, 1, 1, 0, 

Sample:  [1, 1, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1] Energy:  6.0 Occurrences:  1
Sample:  [1, 0, 1, 0, 0, 1, 1, 1, 0, 1, 0, 0, 1, 0, 0, 1] Energy:  6.0 Occurrences:  1
Sample:  [1, 1, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 1, 1, 0, 1] Energy:  7.0 Occurrences:  1
Sample:  [1, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 1, 1, 1, 1] Energy:  9.0 Occurrences:  1
Sample:  [1, 0, 1, 0, 0, 1, 1, 0, 1, 1, 0, 0, 1, 0, 0, 1] Energy:  9.0 Occurrences:  1
Sample:  [0, 1, 0, 0, 1, 0, 1, 1, 0, 1, 0, 1, 1, 0, 1, 0] Energy:  9.0 Occurrences:  1
Sample:  [1, 0, 1, 1, 0, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1] Energy:  9.0 Occurrences:  1
Sample:  [1, 0, 1, 1, 0, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1] Energy:  9.0 Occurrences:  1
Sample:  [1, 0, 1, 0, 0, 1, 1, 0, 1, 1, 0, 0, 1, 0, 0, 1] Energy:  9.0 Occurrences:  1
Sample:  [1, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 1, 1, 0, 0, 1] Energy:  9.0 Occurrences:  1
Sample:  [1, 0, 1, 0, 0, 1, 1, 0, 1, 1, 0, 0, 1, 1, 0, 1] Energy:  10.0 Occurrences:  1
Sample:  [1, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0,

Sample:  [1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1] Energy:  12.0 Occurrences:  1
Sample:  [0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 1, 1] Energy:  15.0 Occurrences:  1
Sample:  [0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 1, 1] Energy:  15.0 Occurrences:  3
Sample:  [0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 1, 1] Energy:  15.0 Occurrences:  1
Sample:  [0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 1, 1] Energy:  15.0 Occurrences:  1
Sample:  [0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 1, 1] Energy:  15.0 Occurrences:  1
Sample:  [0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 1, 1] Energy:  15.0 Occurrences:  1
Sample:  [0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 1, 1] Energy:  15.0 Occurrences:  4
Sample:  [0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 1, 1] Energy:  15.0 Occurrences:  1
Sample:  [1, 0, 1, 0, 1, 0, 1, 1, 0, 1, 0, 1, 0, 0, 1, 1] Energy:  15.0 Occurrences:  1
Sample:  [0, 1, 1, 1, 0, 1, 1, 1, 0, 0, 0, 0, 1, 0, 1, 1] Energy:  18.0 Occurrences:  1
Sample:  [0, 1, 1, 1, 0, 1, 1, 1

Sample:  [1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1] Energy:  28.0 Occurrences:  1
Sample:  [1, 1, 0, 0, 1, 0, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1] Energy:  28.0 Occurrences:  1
Sample:  [1, 1, 0, 1, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1] Energy:  28.0 Occurrences:  1
Sample:  [1, 1, 1, 0, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1] Energy:  31.0 Occurrences:  1
Sample:  [1, 1, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 1, 0, 1, 1] Energy:  31.0 Occurrences:  1
Sample:  [1, 1, 1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 1, 1, 1] Energy:  31.0 Occurrences:  1
Sample:  [0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1] Energy:  31.0 Occurrences:  1
Sample:  [1, 1, 1, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0, 1, 1] Energy:  31.0 Occurrences:  1
Sample:  [1, 1, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 1, 0, 1, 1] Energy:  31.0 Occurrences:  1
Sample:  [1, 1, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 1, 0, 1, 1] Energy:  31.0 Occurrences:  1
Sample:  [1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 0, 0, 0, 0, 1, 1] Energy:  33.0 Occurrences:  1
Sample:  [1, 1, 1, 0, 1, 1, 0, 1

0

In [24]:
num_reads = 10
find_valid_y_1_check(df, num_reads)

Sample:  [0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 1, 1, 0] Energy:  21.0 Occurrences:  1
Sample:  [0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0] Energy:  26.0 Occurrences:  1
Sample:  [0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0, 1, 1, 0] Energy:  27.0 Occurrences:  1
Sample:  [0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0] Energy:  29.0 Occurrences:  2
Sample:  [0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0] Energy:  29.0 Occurrences:  1
Sample:  [0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 0, 1, 0, 0, 0] Energy:  30.0 Occurrences:  1
Sample:  [1, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0] Energy:  38.0 Occurrences:  2
Sample:  [0, 0, 1, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 1, 1, 0] Energy:  39.0 Occurrences:  1
Sample:  [0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0] Energy:  13.0 Occurrences:  2
Sample:  [0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 1, 1, 1, 0] Energy:  14.0 Occurrences:  1
Sample:  [0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0] Energy:  15.0 Occurrences:  1
Sample:  [0, 0, 0, 0, 0, 1, 0, 0

Sample:  [1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1] Energy:  13.0 Occurrences:  1
Sample:  [1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1] Energy:  14.0 Occurrences:  1
Sample:  [0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1] Energy:  15.0 Occurrences:  1
Sample:  [0, 0, 1, 0, 0, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1] Energy:  16.0 Occurrences:  1
Sample:  [1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1] Energy:  18.0 Occurrences:  2
Sample:  [1, 1, 1, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 1, 1, 1] Energy:  18.0 Occurrences:  1
Sample:  [1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1] Energy:  20.0 Occurrences:  1
Sample:  [0, 0, 0, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1] Energy:  21.0 Occurrences:  1
Sample:  [0, 0, 0, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0] Energy:  23.0 Occurrences:  1
Sample:  [1, 1, 0, 1, 0, 0, 1, 1, 0, 1, 0, 1, 0, 1, 1, 1] Energy:  15.0 Occurrences:  1
Sample:  [1, 1, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 1, 1] Energy:  17.0 Occurrences:  1
Sample:  [1, 1, 1, 0, 0, 0, 1, 1

0

In [21]:
num_reads = 100
find_valid_y_2(df, num_reads)#chain_break_fractionはtrue

({0: [],
  1: [],
  2: [],
  3: [],
  4: [],
  5: [],
  6: [],
  7: [],
  8: [],
  9: [],
  10: [],
  11: [],
  12: [],
  13: [],
  14: []},
 {0: 0,
  1: 0,
  2: 0,
  3: 0,
  4: 0,
  5: 0,
  6: 0,
  7: 0,
  8: 0,
  9: 0,
  10: 0,
  11: 0,
  12: 0,
  13: 0,
  14: 0})

In [43]:
num_reads = 100
find_valid_y_20(df, num_reads)#chain_break_fractionはtrue

Sample:  [0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0] Energy:  20.0 Occurrences:  1
Sample:  [0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 1, 1, 0, 1, 0, 0] Energy:  21.0 Occurrences:  1
Sample:  [0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 1] Energy:  21.0 Occurrences:  2
Sample:  [0, 0, 0, 1, 1, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0] Energy:  21.0 Occurrences:  1
Sample:  [0, 0, 0, 1, 1, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0] Energy:  21.0 Occurrences:  1
Sample:  [0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1] Energy:  21.0 Occurrences:  1
Sample:  [0, 0, 0, 1, 1, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0] Energy:  21.0 Occurrences:  2
Sample:  [0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 1] Energy:  21.0 Occurrences:  1
Sample:  [0, 0, 0, 1, 1, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0] Energy:  21.0 Occurrences:  1
Sample:  [0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0] Energy:  22.0 Occurrences:  1
Sample:  [0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1] Energy:  22.0 Occurrences:  1
Sample:  [0, 0, 0, 0, 1, 0, 0, 0

Sample:  [0, 0, 1, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0] Energy:  10.0 Occurrences:  1
Sample:  [1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0] Energy:  10.0 Occurrences:  1
Sample:  [0, 0, 0, 0, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 0, 0] Energy:  11.0 Occurrences:  1
Sample:  [0, 0, 1, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1] Energy:  12.0 Occurrences:  5
Sample:  [0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1, 1, 1] Energy:  12.0 Occurrences:  1
Sample:  [0, 0, 1, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1] Energy:  12.0 Occurrences:  1
Sample:  [0, 0, 1, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1] Energy:  12.0 Occurrences:  1
Sample:  [0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1] Energy:  12.0 Occurrences:  1
Sample:  [1, 1, 0, 0, 1, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0] Energy:  14.0 Occurrences:  1
Sample:  [1, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0] Energy:  14.0 Occurrences:  1
Sample:  [0, 0, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1] Energy:  14.0 Occurrences:  1
Sample:  [1, 1, 0, 0, 0, 0, 1, 0

Sample:  [1, 0, 0, 1, 0, 0, 1, 1, 0, 1, 1, 0, 0, 0, 1, 0] Energy:  5.0 Occurrences:  1
Sample:  [1, 1, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 1, 0] Energy:  5.0 Occurrences:  1
Sample:  [1, 1, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 1, 0] Energy:  5.0 Occurrences:  1
Sample:  [1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0] Energy:  5.0 Occurrences:  1
Sample:  [0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 1, 0] Energy:  5.0 Occurrences:  1
Sample:  [1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 1] Energy:  5.0 Occurrences:  1
Sample:  [0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 1, 0, 0] Energy:  6.0 Occurrences:  1
Sample:  [0, 0, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 1, 1, 0, 0] Energy:  6.0 Occurrences:  1
Sample:  [0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 1, 0, 0] Energy:  6.0 Occurrences:  1
Sample:  [1, 0, 0, 0, 0, 1, 1, 1, 0, 1, 1, 0, 0, 0, 0, 1] Energy:  6.0 Occurrences:  1
Sample:  [1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 0] Energy:  6.0 Occurrences:  1
Sample:  [1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 

Sample:  [1, 0, 0, 0, 1, 0, 1, 0, 1, 1, 1, 0, 0, 1, 1, 0] Energy:  1.0 Occurrences:  1
Sample:  [0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 1, 0, 1, 0, 1] Energy:  1.0 Occurrences:  1
Sample:  [1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 0, 1, 1, 1, 0] Energy:  2.0 Occurrences:  1
Sample:  [1, 0, 0, 0, 1, 0, 0, 1, 1, 0, 1, 0, 1, 1, 1, 0] Energy:  2.0 Occurrences:  1
Sample:  [0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1] Energy:  2.0 Occurrences:  1
Sample:  [1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 0, 1, 1, 1, 0] Energy:  2.0 Occurrences:  1
Sample:  [1, 0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 0, 1, 1, 0] Energy:  2.0 Occurrences:  1
Sample:  [1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 0, 0, 1, 1, 0] Energy:  2.0 Occurrences:  2
Sample:  [1, 1, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 1, 1, 0] Energy:  2.0 Occurrences:  2
Sample:  [0, 0, 1, 1, 0, 0, 1, 1, 0, 1, 0, 1, 0, 1, 0, 1] Energy:  2.0 Occurrences:  1
Sample:  [0, 1, 0, 1, 0, 0, 1, 1, 0, 0, 0, 1, 1, 0, 0, 1] Energy:  3.0 Occurrences:  1
Sample:  [0, 1, 0, 1, 0, 0, 1, 1, 0, 0, 0, 

Sample:  [0, 0, 0, 1, 0, 0, 1, 0, 1, 1, 0, 1, 1, 1, 0, 1] Energy:  3.0 Occurrences:  2
Sample:  [0, 0, 0, 1, 0, 1, 1, 0, 1, 1, 0, 0, 1, 1, 0, 1] Energy:  3.0 Occurrences:  1
Sample:  [0, 0, 0, 1, 0, 0, 1, 0, 1, 1, 0, 1, 1, 1, 0, 1] Energy:  3.0 Occurrences:  1
Sample:  [0, 0, 0, 1, 0, 0, 1, 0, 1, 1, 0, 1, 1, 1, 0, 1] Energy:  3.0 Occurrences:  1
Sample:  [0, 0, 0, 1, 0, 0, 1, 0, 1, 1, 0, 1, 1, 1, 0, 1] Energy:  3.0 Occurrences:  1
Sample:  [0, 0, 0, 1, 0, 0, 1, 0, 1, 1, 0, 1, 1, 1, 0, 1] Energy:  3.0 Occurrences:  1
Sample:  [0, 0, 0, 1, 0, 1, 1, 0, 1, 1, 0, 0, 1, 1, 0, 1] Energy:  3.0 Occurrences:  1
Sample:  [0, 0, 0, 1, 0, 0, 1, 0, 1, 1, 0, 1, 1, 1, 0, 1] Energy:  3.0 Occurrences:  1
Sample:  [0, 0, 0, 1, 0, 0, 1, 0, 1, 1, 0, 1, 1, 1, 0, 1] Energy:  3.0 Occurrences:  1
Sample:  [0, 0, 0, 1, 0, 1, 1, 0, 1, 1, 0, 0, 1, 1, 0, 1] Energy:  3.0 Occurrences:  1
Sample:  [0, 0, 0, 1, 0, 1, 1, 0, 1, 1, 0, 0, 1, 1, 0, 1] Energy:  3.0 Occurrences:  1
Sample:  [0, 0, 0, 1, 0, 0, 1, 0, 1, 1, 0, 

Sample:  [1, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 1, 0, 1, 1] Energy:  6.0 Occurrences:  1
Sample:  [1, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 1, 0, 1, 1] Energy:  6.0 Occurrences:  1
Sample:  [1, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 1, 0, 1, 1] Energy:  6.0 Occurrences:  2
Sample:  [1, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 1, 0, 1, 1] Energy:  6.0 Occurrences:  1
Sample:  [1, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 1, 0, 1, 1] Energy:  6.0 Occurrences:  1
Sample:  [1, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 1, 0, 1, 1] Energy:  6.0 Occurrences:  1
Sample:  [1, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 1, 0, 1, 1] Energy:  6.0 Occurrences:  1
Sample:  [1, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 1, 1] Energy:  6.0 Occurrences:  1
Sample:  [1, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 1, 0, 1, 1] Energy:  6.0 Occurrences:  1
Sample:  [1, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 1, 0, 1, 1] Energy:  6.0 Occurrences:  1
Sample:  [1, 1, 0, 0, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1] Energy:  6.0 Occurrences:  1
Sample:  [1, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 

Sample:  [1, 1, 1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 1, 1, 1] Energy:  15.0 Occurrences:  1
Sample:  [1, 1, 0, 0, 0, 1, 1, 1, 0, 1, 0, 1, 1, 0, 0, 1] Energy:  15.0 Occurrences:  1
Sample:  [1, 1, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1] Energy:  18.0 Occurrences:  1
Sample:  [1, 1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 1, 1, 0, 0, 1] Energy:  18.0 Occurrences:  1
Sample:  [1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 1, 1] Energy:  18.0 Occurrences:  1
Sample:  [1, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 1, 1] Energy:  18.0 Occurrences:  1
Sample:  [0, 1, 0, 1, 0, 1, 1, 1, 0, 0, 0, 1, 1, 0, 1, 1] Energy:  18.0 Occurrences:  1
Sample:  [1, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 1, 1] Energy:  18.0 Occurrences:  1
Sample:  [1, 1, 1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0] Energy:  18.0 Occurrences:  1
Sample:  [0, 1, 0, 1, 0, 1, 1, 0, 0, 1, 0, 1, 1, 0, 1, 1] Energy:  18.0 Occurrences:  1
Sample:  [1, 1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 1, 1, 0, 0, 1] Energy:  18.0 Occurrences:  1
Sample:  [1, 1, 0, 1, 0, 0, 0, 0

Sample:  [1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 0, 1, 0, 0, 1, 1] Energy:  28.0 Occurrences:  1
Sample:  [1, 1, 0, 1, 0, 0, 1, 1, 0, 1, 0, 1, 0, 0, 1, 1] Energy:  28.0 Occurrences:  1
Sample:  [1, 1, 0, 1, 0, 0, 1, 1, 1, 1, 0, 1, 0, 0, 1, 1] Energy:  28.0 Occurrences:  2
Sample:  [1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1] Energy:  31.0 Occurrences:  1
Sample:  [1, 1, 0, 1, 0, 0, 1, 1, 0, 1, 0, 1, 0, 1, 1, 1] Energy:  31.0 Occurrences:  1
Sample:  [1, 0, 0, 1, 0, 0, 1, 1, 1, 1, 0, 1, 0, 0, 1, 1] Energy:  31.0 Occurrences:  1
Sample:  [1, 0, 0, 1, 0, 0, 1, 1, 1, 1, 0, 1, 0, 0, 1, 1] Energy:  31.0 Occurrences:  1
Sample:  [1, 0, 0, 1, 0, 0, 1, 1, 1, 1, 0, 1, 0, 0, 1, 1] Energy:  31.0 Occurrences:  1
Sample:  [1, 1, 0, 1, 0, 0, 1, 1, 0, 1, 0, 1, 0, 1, 1, 1] Energy:  31.0 Occurrences:  1
Sample:  [0, 1, 1, 0, 1, 1, 1, 1, 0, 1, 0, 0, 1, 0, 1, 1] Energy:  33.0 Occurrences:  1
Sample:  [0, 1, 1, 0, 0, 1, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1] Energy:  33.0 Occurrences:  1
Sample:  [1, 0, 1, 1, 0, 0, 1, 1

0

In [22]:
num_reads = 100
find_valid_y_3(df, num_reads, 100)#chain_strength=9のときが最多

({0: [],
  1: [],
  2: [],
  3: [[1, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 0, 1]],
  4: [[0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1, 0, 1, 1, 1],
   [0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1, 0, 1, 1, 1],
   [0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1, 0, 1, 1, 1],
   [0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1, 0, 1, 1, 1],
   [0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1, 0, 1, 1, 1],
   [0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1, 0, 1, 1, 1],
   [0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1, 1, 0, 1, 1, 1]],
  5: [],
  6: [[0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1]],
  7: [],
  8: [],
  9: [[1, 0, 1, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 1, 0, 1],
   [1, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 1, 0, 1, 0, 1]],
  10: [],
  11: [[1, 0, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 1]],
  12: [[1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1],
   [1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 1, 1, 1],
   [1, 0, 0, 0, 1, 1, 1, 0, 0, 1, 1, 0, 0, 1, 0, 1]],
  13: [],
  14: [[1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 1, 0, 1, 1, 1]],
  15: [],
  16: [[1, 1, 1, 0, 0, 0, 1, 0, 0, 1,

In [51]:
num_reads = 10
find_valid_y_3_check(df, num_reads, 100)

Sample:  [1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0] Energy:  20.0 Occurrences:  1
Sample:  [1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0] Energy:  20.0 Occurrences:  2
Sample:  [1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0] Energy:  20.0 Occurrences:  1
Sample:  [1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0] Energy:  20.0 Occurrences:  1
Sample:  [1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0] Energy:  28.0 Occurrences:  1
Sample:  [1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0] Energy:  28.0 Occurrences:  1
Sample:  [1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0] Energy:  28.0 Occurrences:  1
Sample:  [1, 1, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1] Energy:  37.0 Occurrences:  1
Sample:  [1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0] Energy:  41.0 Occurrences:  1
Sample:  [1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0] Energy:  30.0 Occurrences:  1
Sample:  [1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0] Energy:  30.0 Occurrences:  1
Sample:  [0, 0, 1, 1, 0, 0, 0, 0

Sample:  [0, 0, 1, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0] Energy:  21.0 Occurrences:  1
Sample:  [0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0] Energy:  22.0 Occurrences:  1
Sample:  [1, 1, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0] Energy:  22.0 Occurrences:  1
Sample:  [0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0] Energy:  25.0 Occurrences:  1
Sample:  [0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0] Energy:  27.0 Occurrences:  1
Sample:  [0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 1, 1, 0, 1, 0, 0] Energy:  27.0 Occurrences:  1
Sample:  [0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0] Energy:  29.0 Occurrences:  1
Sample:  [0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0] Energy:  29.0 Occurrences:  1
Sample:  [1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 1] Energy:  30.0 Occurrences:  1
Sample:  [1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1] Energy:  33.0 Occurrences:  1
Sample:  [0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0] Energy:  19.0 Occurrences:  1
Sample:  [0, 0, 1, 0, 0, 1, 1, 0

Sample:  [0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1, 0, 1, 0, 1] Energy:  22.0 Occurrences:  1
Sample:  [1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0] Energy:  26.0 Occurrences:  1
Sample:  [1, 0, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0] Energy:  26.0 Occurrences:  1
Sample:  [0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0] Energy:  27.0 Occurrences:  1
Sample:  [1, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0] Energy:  30.0 Occurrences:  1
Sample:  [1, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1] Energy:  33.0 Occurrences:  1
Sample:  [1, 0, 1, 1, 0, 1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 1] Energy:  37.0 Occurrences:  1
Sample:  [0, 0, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 0, 1, 1, 1] Energy:  38.0 Occurrences:  1
Sample:  [0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1] Energy:  40.0 Occurrences:  1
Sample:  [0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0] Energy:  44.0 Occurrences:  1
Sample:  [1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1] Energy:  22.0 Occurrences:  1
Sample:  [1, 1, 0, 0, 0, 0, 0, 0

Sample:  [0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 1, 0, 1, 0, 0] Energy:  24.0 Occurrences:  1
Sample:  [1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 0, 1, 0] Energy:  29.0 Occurrences:  1
Sample:  [0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 1, 0, 0] Energy:  29.0 Occurrences:  1
Sample:  [1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0] Energy:  30.0 Occurrences:  1
Sample:  [0, 0, 0, 0, 1, 1, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0] Energy:  31.0 Occurrences:  1
Sample:  [1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 1, 0, 1, 1, 0] Energy:  38.0 Occurrences:  1
Sample:  [0, 0, 1, 0, 0, 1, 1, 1, 1, 0, 1, 0, 0, 1, 1, 0] Energy:  38.0 Occurrences:  1
Sample:  [0, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 1, 1, 0] Energy:  38.0 Occurrences:  1
Sample:  [0, 0, 1, 0, 0, 1, 0, 1, 1, 1, 0, 0, 1, 1, 0, 0] Energy:  41.0 Occurrences:  1
Sample:  [0, 0, 1, 0, 1, 0, 1, 1, 1, 1, 0, 0, 0, 0, 1, 0] Energy:  52.0 Occurrences:  1
Sample:  [0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0] Energy:  21.0 Occurrences:  1
Sample:  [0, 0, 1, 1, 0, 1, 0, 0

Sample:  [0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1, 1, 0] Energy:  20.0 Occurrences:  1
Sample:  [0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0, 1, 1, 0, 0] Energy:  21.0 Occurrences:  1
Sample:  [0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 1, 1, 0] Energy:  21.0 Occurrences:  1
Sample:  [0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1] Energy:  30.0 Occurrences:  1
Sample:  [1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1] Energy:  30.0 Occurrences:  1
Sample:  [0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 1, 1, 0, 1] Energy:  30.0 Occurrences:  1
Sample:  [1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0] Energy:  33.0 Occurrences:  1
Sample:  [0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0] Energy:  33.0 Occurrences:  1
Sample:  [0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 1] Energy:  37.0 Occurrences:  1
Sample:  [1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0] Energy:  41.0 Occurrences:  1
Sample:  [0, 1, 0, 0, 1, 1, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0] Energy:  21.0 Occurrences:  1
Sample:  [0, 0, 1, 0, 0, 1, 0, 0

...

Sample:  [1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1] Energy:  1.0 Occurrences:  1
Sample:  [1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1] Energy:  2.0 Occurrences:  1
Sample:  [1, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1] Energy:  4.0 Occurrences:  1
Sample:  [1, 0, 1, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 1, 1, 1] Energy:  4.0 Occurrences:  1
Sample:  [0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 1, 1] Energy:  4.0 Occurrences:  1
Sample:  [0, 1, 0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1] Energy:  6.0 Occurrences:  1
Sample:  [0, 0, 0, 1, 0, 1, 0, 1, 1, 0, 1, 1, 0, 0, 0, 0] Energy:  9.0 Occurrences:  1
Sample:  [0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 1, 0, 0, 0] Energy:  10.0 Occurrences:  1
Sample:  [0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 1, 1, 0, 0, 0] Energy:  11.0 Occurrences:  1
Sample:  [0, 1, 0, 1, 1, 0, 0, 1, 1, 1, 0, 1, 1, 0, 0, 0] Energy:  21.0 Occurrences:  1
Sample:  [0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 1, 1, 1, 0, 0] Energy:  3.0 Occurrences:  1
Sample:  [1, 0, 0, 1, 0, 1, 1, 1, 0, 1, 

Sample:  [1, 1, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0] Energy:  2.0 Occurrences:  1
Sample:  [1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0] Energy:  2.0 Occurrences:  1
Sample:  [1, 1, 0, 0, 0, 1, 1, 1, 0, 1, 1, 0, 0, 1, 0, 0] Energy:  2.0 Occurrences:  1
Sample:  [1, 0, 0, 0, 1, 0, 1, 1, 1, 1, 0, 0, 1, 1, 0, 0] Energy:  3.0 Occurrences:  1
Sample:  [1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0] Energy:  3.0 Occurrences:  1
Sample:  [0, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 0, 0, 1, 0, 0] Energy:  4.0 Occurrences:  1
Sample:  [1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0] Energy:  5.0 Occurrences:  1
Sample:  [1, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 1] Energy:  7.0 Occurrences:  1
Sample:  [0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 1, 1, 0] Energy:  15.0 Occurrences:  1
Sample:  [0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0] Energy:  18.0 Occurrences:  1
Sample:  [1, 1, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 1] Energy:  0.0 Occurrences:  1
Sample:  [0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0

Sample:  [0, 1, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 1, 1, 1] Energy:  4.0 Occurrences:  1
Sample:  [0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0] Energy:  6.0 Occurrences:  1
Sample:  [1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 1, 1, 0, 0, 1] Energy:  6.0 Occurrences:  1
Sample:  [1, 1, 0, 1, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0] Energy:  6.0 Occurrences:  1
Sample:  [1, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0] Energy:  11.0 Occurrences:  1
Sample:  [0, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 0, 0] Energy:  12.0 Occurrences:  1
Sample:  [0, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0] Energy:  12.0 Occurrences:  1
Sample:  [1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0] Energy:  14.0 Occurrences:  1
Sample:  [1, 1, 0, 1, 1, 0, 0, 1, 1, 1, 1, 1, 0, 0, 1, 0] Energy:  33.0 Occurrences:  1
Sample:  [1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 0, 1, 1, 0, 0, 0] Energy:  35.0 Occurrences:  1
Sample:  [1, 1, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 1] Energy:  0.0 Occurrences:  1
Sample:  [0, 0, 0, 0, 1, 0, 1, 1, 1, 

Sample:  [1, 1, 0, 1, 0, 1, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0] Energy:  3.0 Occurrences:  1
Sample:  [0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 1, 1, 1, 1, 1, 0] Energy:  5.0 Occurrences:  1
Sample:  [0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1] Energy:  6.0 Occurrences:  1
Sample:  [1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0] Energy:  6.0 Occurrences:  1
Sample:  [0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 1, 1, 0] Energy:  7.0 Occurrences:  1
Sample:  [0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0] Energy:  7.0 Occurrences:  1
Sample:  [0, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1] Energy:  14.0 Occurrences:  1
Sample:  [1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0] Energy:  18.0 Occurrences:  1
Sample:  [1, 0, 1, 0, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 1, 1] Energy:  23.0 Occurrences:  1
Sample:  [1, 1, 0, 1, 1, 1, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0] Energy:  23.0 Occurrences:  1
Sample:  [1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0] Energy:  3.0 Occurrences:  1
Sample:  [0, 0, 1, 1, 0, 1, 0, 1, 0, 1,

OSError: ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response'))

In [141]:
def test_validity():
    df1 = pd.read_csv('../../input/ost6.csv', sep=',',index_col=0)
    df2 = pd.read_csv('../../input/ost6.csv', sep=',',index_col=0)
    new_y = np.array([0, 1, 0, 0, 0, 1])
    df2['Y'] = new_y
    t_list1 = calc_marginals(df1)
    t_list2 = calc_marginals(df2)
    print(t_list1)
    print(t_list2)
    assert np.all(t_list1[[0,2,3]] == t_list2[[0,2,3]]) 
test_validity()

[2 2 0 1]
[2 2 1 2]


AssertionError: 

In [72]:
def time_measurement(df):
    sum_time = 0
    annealing_time = 20
    for t1 in range(0, sum(df['LI'])+1):
        timeit_repeat = timeit.repeat("make_Hamiltonian(df, t1)", number=1, repeat=1, globals={"make_Hamiltonian": make_Hamiltonian, "df": df, "t1": t1})
        sum_time += timeit_repeat[0] + annealing_time
    return sum_time

In [73]:
#==========
#テストコード
#==========
def test_find_valid_y():
    df = pd.read_csv('../../input/ost6.csv', sep=',', index_col=0)
    true_t1 = sum(df['Y'] * df['LI'])
    valid_y_list, valid_y_num = find_valid_y(df,  num_reads=20)
    print(valid_y_list, valid_y_num)
    assert valid_y_num[true_t1] > 0
    
test_find_valid_y()

{0: [], 1: [], 2: [[1, 0, 0, 0, 0, 1], [1, 0, 0, 0, 1, 0]], 3: [], 4: [], 5: [], 6: []} {0: 0, 1: 0, 2: 2, 3: 0, 4: 0, 5: 0, 6: 0}


In [64]:
def test_validity():
    df1 = pd.read_csv('../../input/ost6.csv', sep=',',index_col=0)
    df2 = pd.read_csv('../../input/ost6.csv', sep=',',index_col=0)
    new_y = np.array([1, 0, 0, 0, 0, 1])
    df2['Y'] = new_y
    t_list1 = calc_marginals(df1)
    t_list2 = calc_marginals(df2)
    print(t_list1)
    print(t_list2)
    assert np.all(t_list1[[0,2,3]] == t_list2[[0,2,3]]) 

#test_validity()

[2 2 0 1]
[2 2 0 1]
